In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import ml.generators.mip_generator as generator
import tensorflow as tf

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
TrainGen = generator.MipGenerator
ValGen = generator.MipGenerator
train_gen = TrainGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

In [6]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

In [7]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(220, 220, 3))
for layer in base_model.layers:
    layer.trainable = False
finished_model = add_new_last_layer(base_model, 1)
finished_model.compile(optimizer=adam(lr=0.01),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
finished_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 220, 220, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 220, 220, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 220, 220, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 110, 110, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 110, 110, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 110, 110, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 55, 55, 128)       0         
__________

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
mc_callback = ModelCheckpoint(filepath='tmp/vgg16_pretrained_weights.hdf5', verbose=1)
finished_model.fit_generator(
    generator=train_gen.generate(),
    steps_per_epoch=train_gen.get_steps_per_epoch(),
    validation_data=val_gen.generate(),
    validation_steps=val_gen.get_steps_per_epoch(),
    epochs=20,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)

Epoch 1/20
 4/87 [>.............................] - ETA: 4:55 - loss: 4.6598 - acc: 0.5469

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


87/87 [==============================] - 350s 4s/step - loss: 6.6508 - acc: 0.5754 - val_loss: 4.9820 - val_acc: 0.6875

Epoch 00001: saving model to tmp/vgg16_pretrained_weights.hdf5
Epoch 2/20
86/87 [============================>.] - ETA: 3s - loss: 6.6967 - acc: 0.5799